# Component Swapping Experiments

## Setup

In [1]:
from IPython import get_ipython
from IPython.display import clear_output, display

ipython = get_ipython()
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

/tmp/ipykernel_2703113/1711627342.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_2703113/1711627342.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
import os
from typing import List, Optional, Union, Dict, Tuple

import torch
from torch import Tensor
import numpy as np
import einops
from fancy_einsum import einsum
import circuitsvis as cv

import transformer_lens.utils as utils

from transformer_lens import HookedTransformer
import transformer_lens.patching as patching

from transformers import AutoModelForCausalLM

from torch import Tensor
from jaxtyping import Float
import plotly.express as px

from functools import partial

from torchtyping import TensorType as TT

from path_patching_cm.path_patching import Node, IterNode, path_patch, act_patch
from path_patching_cm.ioi_dataset import IOIDataset, NAMES
from neel_plotly import imshow as imshow_n

from utils.visualization import imshow_p, plot_attention_heads
from utils.data_utils import generate_data_and_caches
from utils.metrics import _logits_to_mean_logit_diff
from utils.visualization import (
    plot_attention_heads,
    scatter_attention_and_contribution,
    get_attn_head_patterns
)

from utils.circuit_utils import (
    ComponentDict,
    get_components_to_swap,
    load_swapped_params
)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [3]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def two_lines(tensor1, tensor2, renderer=None, **kwargs):
    px.line(y=[utils.to_numpy(tensor1), utils.to_numpy(tensor2)], **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [4]:
torch.set_grad_enabled(False)

In [5]:
from utils.data_utils import UniversalPatchingDataset
from utils.circuit_utils import run_with_batches, get_data_and_metrics

In [6]:

model = HookedTransformer.from_pretrained(
        "EleutherAI/pythia-160m",
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        refactor_factored_attn_matrices=False,
    )
model.set_use_hook_mlp_in(True)
model.tokenizer.add_bos_token = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


In [8]:
ds, metric = get_data_and_metrics(model, 'ioi')

In [9]:
ds = UniversalPatchingDataset.from_ioi(model, 100)
batch_size = 100

In [10]:
clean_logits = run_with_batches(model, ds.toks, batch_size, ds.max_seq_len)

In [13]:
from utils.metrics import compute_logit_diff

compute_logit_diff(clean_logits, ds.answer_toks, positions=ds.positions, flags_tensor=ds.group_flags)

tensor(4.1922, device='cuda:0')

## Model Setup

In [14]:
from typing import Callable

def get_ioi_logit_diff_for_model(model: HookedTransformer):
    N = 70
    model.tokenizer.add_bos_token = False
    ioi_dataset, abc_dataset = generate_data_and_caches(model, N, verbose=True)
    clean_logits, clean_cache = model.run_with_cache(ioi_dataset.toks)
    return _logits_to_mean_logit_diff(clean_logits, ioi_dataset)
    

def get_gt_prob_diff_for_model(model: HookedTransformer):
    N = 1000
    ds = UniversalPatchingDataset.from_greater_than(model, N)
    batch_size = 100
    logits = run_with_batches(model, ds.toks, batch_size, ds.max_seq_len)
    return compute_probability_diff(logits, ds.answer_toks, positions=ds.positions, flags_tensor=ds.group_flags, mode="group_sum")


def run_swap_experiment_test(
    model_name: str,
    dataset_eval_fn: Callable, 
    source_checkpoint: int, 
    target_checkpoint: int, 
    heads: List[Tuple[int, int]],
    include_ln: bool = False,
    include_mlps: bool = False,
    cache_dir: str = "/mnt/hdd-0/circuits-over-time/model_cache/ct"
    ):

    # get baseline logit diff metrics
    model = HookedTransformer.from_pretrained(
        "EleutherAI/pythia-160m",
        checkpoint_value=target_checkpoint,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        refactor_factored_attn_matrices=False,
    )
    model.set_use_hook_mlp_in(True)
    model.tokenizer.add_bos_token = False

    baseline_metric = dataset_eval_fn(model)

    source_model = AutoModelForCausalLM.from_pretrained(
        model_name, revision=f"step{source_checkpoint}", cache_dir=cache_dir
    )

    target_model = AutoModelForCausalLM.from_pretrained(
        model_name, revision=f"step{target_checkpoint}", cache_dir=cache_dir
    )

    component_dict = ComponentDict(layer_heads=heads, include_ln=include_ln, include_mlps=include_mlps)

    component_params = get_components_to_swap(source_model, component_dict, cache_dir)
    load_swapped_params(target_model, component_params)

    model = HookedTransformer.from_pretrained(
        "EleutherAI/pythia-160m",
        hf_model=target_model,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        refactor_factored_attn_matrices=False,
    )
    model.set_use_hook_mlp_in(True)
    model.tokenizer.add_bos_token = False

    swapped_metric = dataset_eval_fn(model)

    return baseline_metric.item(), swapped_metric.item()

In [15]:
import pandas as pd

def run_swap_grid_search(
    model_name: str,
    dataset_eval_fn: Callable,
    source_range: Tuple[int, int],
    target_range: Tuple[int, int],
    heads: List[Tuple[int, int]]
    ) -> pd.DataFrame:
    
    # Initialize an empty list to store the results
    results = []

    # Loop through the target and source ranges
    for target in range(target_range[0], target_range[1], 1000):
        for source in range(max(source_range[0], (target+1000)), source_range[1], 1000):
            
            # Run the experiment to get baseline and swapped values
            baseline_ld, swapped_ld = run_swap_experiment_test(model_name, dataset_eval_fn, source, target, heads=heads)
            
            # Calculate the difference
            difference = swapped_ld - baseline_ld
            
            # Append the results as a dictionary
            results.append({
                "source": source,
                "target": target,
                "baseline_val": baseline_ld,
                "swapped_val": swapped_ld,
                "difference": difference
            })
            
            print(f"Source: {source} -> Target: {target}")

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

### IOI

In [27]:
model_name = "EleutherAI/pythia-160m"
whole_circuit = [(4, 6), (4, 11), (6, 6), (7, 2), (7, 9), (8, 9), (8, 2), (8, 10)]
nmh = [(8, 2), (8, 10)]
top_3 = nmh + [(8, 9)]
s2i = [(6, 6), (7, 2), (7, 9)]
csh = [(8, 9)]
idh = [(4, 6), (4, 11)]

In [44]:
results = run_swap_grid_search(model_name, get_ioi_logit_diff_for_model, source_range=(4000, 20000), target_range=(1000, 10000), heads=whole_circuit)
results['pct_change'] = results['difference'] / results['baseline_val']

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


Source: 4000 -> Target: 1000


...

Source: 18000 -> Target: 9000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer
Source: 19000 -> Target: 9000


In [45]:
results.head(50)

,source,target,baseline_val,swapped_val,difference,pct_change
0,4000,1000,-0.302210,-0.315800,-0.013590,0.044967
1,5000,1000,-0.302210,-0.316972,-0.014762,0.048848
2,6000,1000,-0.302210,-0.316227,-0.014017,0.046382
3,7000,1000,-0.302210,-0.310158,-0.007948,0.026300
4,8000,1000,-0.302210,-0.308751,-0.006541,0.021645
5,9000,1000,-0.302210,-0.306486,-0.004276,0.014150
6,10000,1000,-0.302210,-0.310836,-0.008626,0.028543
7,11000,1000,-0.302210,-0.309256,-0.007046,0.023314
8,12000,1000,-0.302210,-0.313383,-0.011173,0.036972
9,13000,1000,-0.302210,-0.307827,-0.005617,0.018588


In [46]:
sorted_df = results.sort_values(by='difference', ascending=False)
top_n = sorted_df.head(10)
print(top_n)

    source  target  baseline_val  swapped_val  difference  pct_change
34    6000    3000      0.047282     0.299044    0.251762    5.324676
33    5000    3000      0.047282     0.268404    0.221122    4.676648
35    7000    3000      0.047282     0.250925    0.203643    4.306972
36    8000    3000      0.047282     0.239457    0.192175    4.064430
49    6000    4000      0.407952     0.547358    0.139406    0.341722
32    4000    3000      0.047282     0.178955    0.131673    2.784835
37    9000    3000      0.047282     0.174952    0.127670    2.700167
48    5000    4000      0.407952     0.514669    0.106718    0.261594
90    8000    7000      1.843825     1.939461    0.095636    0.051868
39   11000    3000      0.047282     0.126258    0.078975    1.670302


In [47]:
# save results
model_shortname = model_name[11:]
os.makedirs("/mnt/hdd-0/circuits-over-time/results/comp_swap", exist_ok=True)
os.makedirs(f"/mnt/hdd-0/circuits-over-time/results/comp_swap/{model_shortname}", exist_ok=True)
results.to_csv(f"/mnt/hdd-0/circuits-over-time/results/comp_swap/{model_shortname}/ioi_whole_circuit_logit_diff_results.csv", index=False)

OSError: [Errno 122] Disk quota exceeded

### Greater Than

In [63]:
model_name = "EleutherAI/pythia-160m"
successor_heads = [(11, 9), (4, 10), (4, 6), (5, 0), (5, 9), (9, 0), (10, 4)]
induction_heads = [(4, 11), (5, 0)]
all = successor_heads + induction_heads

In [64]:
results = run_swap_grid_search(model_name, get_gt_prob_diff_for_model, source_range=(2000, 20000), target_range=(1000, 19000), heads=all)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


Source: 2000 -> Target: 1000


...

Source: 19000 -> Target: 17000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer
Source: 19000 -> Target: 18000


In [65]:
results['pct_change'] = results['difference'] / results['baseline_ld']
results = results.rename(columns={'baseline_ld': 'baseline_val', 'swapped_ld': 'swapped_val'})

In [1]:
sorted_df = results.sort_values(by='difference', ascending=False)
top_n = sorted_df.head(10)
print(top_n)

NameError: name 'results' is not defined

In [59]:
results.head(50)

,source,target,baseline_ld,swapped_ld,difference
0,2000,1000,-0.172767,-0.159733,0.013034
1,3000,1000,-0.174749,-0.146568,0.028181
2,4000,1000,-0.177742,-0.130802,0.046940
3,5000,1000,-0.170297,-0.123153,0.047143
4,6000,1000,-0.175155,-0.130347,0.044809
5,7000,1000,-0.176924,-0.135718,0.041205
6,8000,1000,-0.176415,-0.167677,0.008738
7,9000,1000,-0.172683,-0.160158,0.012525
8,10000,1000,-0.172487,-0.170555,0.001933
9,11000,1000,-0.172088,-0.152882,0.019206


In [69]:
# save results
model_shortname = model_name[11:]
os.makedirs("/mnt/hdd-0/circuits-over-time/results/comp_swap", exist_ok=True)
os.makedirs(f"/mnt/hdd-0/circuits-over-time/results/comp_swap/{model_shortname}", exist_ok=True)
results.to_csv(f"/mnt/hdd-0/circuits-over-time/results/comp_swap/{model_shortname}/gt_induction_successor_head_prob_diff_results.csv", index=False)

## Data Setup

In [54]:
model_shortname = model_name[11:]
model_shortname

'pythia-160m'

In [48]:


N = 70
model.tokenizer.add_bos_token = False
ioi_dataset, abc_dataset = generate_data_and_caches(model, N, verbose=True)

In [51]:
clean_logits, clean_cache = model.run_with_cache(ioi_dataset.toks)
corrupted_logits, corrupted_cache = model.run_with_cache(abc_dataset.toks)

clean_logit_diff = _logits_to_mean_logit_diff(clean_logits, ioi_dataset)
print(f"Clean logit diff: {clean_logit_diff:.4f}")

corrupted_logit_diff = _logits_to_mean_logit_diff(corrupted_logits, ioi_dataset)
print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")

Clean logit diff: 1.1939
Corrupted logit diff: -0.7280


In [19]:
CLEAN_BASELINE = clean_logit_diff
CORRUPTED_BASELINE = corrupted_logit_diff

In [20]:
def logit_diff_denoising(
    logits: Float[Tensor, "batch seq d_vocab"],
    dataset: IOIDataset,
    flipped_logit_diff: float = corrupted_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    return_tensor: bool = False,
) -> Float[Tensor, ""]:
    '''
    Linear function of logit diff, calibrated so that it equals 0 when performance is
    same as on flipped input, and 1 when performance is same as on clean input.
    '''
    patched_logit_diff = _logits_to_ave_logit_diff(logits, dataset)
    ld = ((patched_logit_diff - flipped_logit_diff) / (clean_logit_diff  - flipped_logit_diff))
    if return_tensor:
        return ld
    else:
        return ld.item()


def logit_diff_noising(
        logits: Float[Tensor, "batch seq d_vocab"],
        dataset: IOIDataset = ioi_dataset,
        clean_logit_diff: float = clean_logit_diff,
        corrupted_logit_diff: float = corrupted_logit_diff,
        return_tensor: bool = False,
    ) -> float:
        '''
        We calibrate this so that the value is 0 when performance isn't harmed (i.e. same as IOI dataset),
        and -1 when performance has been destroyed (i.e. is same as ABC dataset).
        '''
        patched_logit_diff = _logits_to_ave_logit_diff(logits, dataset)
        ld = ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff))

        if return_tensor:
            return ld
        else:
            return ld.item()

logit_diff_denoising_ioi = partial(logit_diff_denoising, dataset=ioi_dataset)
logit_diff_noising_ioi = partial(logit_diff_noising, dataset=ioi_dataset)

## Tool Setup

### Activation Patching

## Direct Logit Attribution

In [ ]:
sio_answer_tensor = torch.cat(
    (torch.tensor(ioi_dataset.io_tokenIDs).unsqueeze(dim=1), torch.tensor(ioi_dataset.s_tokenIDs).unsqueeze(dim=1)), 
    dim=1
)

In [ ]:
answer_residual_directions: Float[Tensor, "batch 2 d_model"] = model.tokens_to_residual_directions(sio_answer_tensor)
print("Answer residual directions shape:", answer_residual_directions.shape)

correct_residual_directions, incorrect_residual_directions = answer_residual_directions.unbind(dim=1)
logit_diff_directions: Float[Tensor, "batch d_model"] = correct_residual_directions - incorrect_residual_directions
print(f"Logit difference directions shape:", logit_diff_directions.shape)

In [ ]:
# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type].
final_residual_stream: Float[Tensor, "batch seq d_model"] = clean_cache["resid_post", -1]
print(f"Final residual stream shape: {final_residual_stream.shape}")
scaled_residual_stream = clean_cache.apply_ln_to_stack(final_residual_stream, layer=-1)
print(f"Scaled residual stream shape: {scaled_residual_stream.shape}")
scaled_final_token_residual_stream: Float[Tensor, "batch d_model"] = scaled_residual_stream[torch.arange(final_residual_stream.size(0)), ioi_dataset.word_idx["end"]]
print(f"Final token residual stream shape: {scaled_final_token_residual_stream.shape}")

average_logit_diff = einops.einsum(
    scaled_final_token_residual_stream, logit_diff_directions,
    "batch d_model, batch d_model ->"
) / 70

print(f"Calculated average logit diff: {average_logit_diff:.10f}")
print(f"Original logit difference:     {clean_logit_diff:.10f}")

#torch.testing.assert_close(average_logit_diff, clean_logit_diff)

In [ ]:
from transformer_lens import ActivationCache

def residual_stack_to_logit_diff(
    residual_stack: Float[Tensor, "... batch d_model"],
    cache: ActivationCache,
    logit_diff_directions: Float[Tensor, "batch d_model"] = logit_diff_directions,
) -> Float[Tensor, "..."]:
    '''
    Gets the avg logit difference between the correct and incorrect answer for a given
    stack of components in the residual stream.
    '''
    # SOLUTION
    batch_size = residual_stack.size(-2)
    scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer=-1, pos_slice=-1)
    return einops.einsum(
        scaled_residual_stack, logit_diff_directions,
        "... batch d_model, batch d_model -> ..."
    ) / batch_size


# Test function by checking that it gives the same result as the original logit difference
# t.testing.assert_close(
#     residual_stack_to_logit_diff(final_token_residual_stream, cache),
#     original_average_logit_diff
# )

### Logit Lens

In [ ]:
accumulated_residual, labels = clean_cache.accumulated_resid(layer=-1, incl_mid=False, return_labels=True)
accumulated_residual_final_token = accumulated_residual[:, torch.arange(accumulated_residual.size(1)), ioi_dataset.word_idx["end"]]
print(f"Shape of accumulated residual: {accumulated_residual_final_token.shape}")
# accumulated_residual has shape (component, batch, d_model)

logit_lens_logit_diffs: Float[Tensor, "component"] = residual_stack_to_logit_diff(accumulated_residual_final_token, clean_cache)
line(logit_lens_logit_diffs, x=np.arange(model.cfg.n_layers+1), hover_name=labels, title="Logit Difference From Accumulated Residual Stream")

### Layer Attribution

In [ ]:
per_layer_residual, labels = clean_cache.decompose_resid(layer=-1, return_labels=True)
per_layer_residual_final_token = per_layer_residual[:, torch.arange(per_layer_residual.size(1)), ioi_dataset.word_idx["end"]]
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual_final_token, clean_cache)

line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

### Head Attribution

In [ ]:
per_head_residual, labels = clean_cache.stack_head_results(layer=-1, return_labels=True)
print(f"Shape of per head residual: {per_head_residual.shape}")
per_head_residual_final_token = per_head_residual[:, torch.arange(per_head_residual.size(1)), ioi_dataset.word_idx["end"]]
print(f"Shape of per head residual: {per_head_residual_final_token.shape}")
per_head_residual_final_token = einops.rearrange(
    per_head_residual_final_token,
    "(layer head) ... -> layer head ...",
    layer=model.cfg.n_layers
)
print(f"Shape of per head residual: {per_head_residual_final_token.shape}")
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual_final_token, clean_cache)

imshow(per_head_logit_diffs, xaxis="Head", yaxis="Layer", title="Logit Difference From Each Head")

In [ ]:
plot_attention_heads(per_head_logit_diffs/clean_logit_diff, top_n=15, range_x=[0, 1])

In [ ]:
nmh_candidates = [(8, 10), (10, 7)]

In [ ]:
import pandas as pd
def scatter_attention_and_contribution(
    model,
    head,
    prompts,
    end_positions,
    io_positions,
    s_positions,
    answer_residual_directions,
    return_vals=False,
    return_fig=False,
):

    df = []

    layer, head_idx = head
    # Get the attention output to the residual stream for the head
    _, cache = model.run_with_cache(prompts)
    per_head_residual, _ = cache.stack_head_results(
        layer=-1, return_labels=True
    )
    scaled_residual_stack = cache.apply_ln_to_stack(
        per_head_residual, layer=-1
    )
    print(scaled_residual_stack.shape)
    scaled_residual_stack_final_token = scaled_residual_stack[:, torch.arange(scaled_residual_stack.size(1)), end_positions]
    head_resid = scaled_residual_stack_final_token[layer * model.cfg.n_heads + head_idx]

    # Loop over each prompt
    for i in range(len(answer_residual_directions)):
        # Get attention values
        tokens, attn, names = get_attn_head_patterns(model, prompts[i], [head])

        # For IO
        # Get the attention contribution in the residual directions
        dot = einsum(
            "d_model, d_model -> ", head_resid[i], answer_residual_directions[i][0]
        )

        # Get the attention probability to the IO answer
        prob = attn[0, 14, io_positions[i]]
        df.append([prob, dot, "IO", prompts[i]])

        # For S
        # Get the attention contribution in the residual directions
        dot = einsum(
            "d_model, d_model -> ", head_resid[i], answer_residual_directions[i][1]
        )
        # Get the attention probability to the S answer
        prob = attn[0, 14, s_positions[i]]
        df.append([prob, dot, "S", prompts[i]])

    # Plot the results
    viz_df = pd.DataFrame(
        df, columns=[f"Attn Prob on Name", f"Dot w Name Embed", "Name Type", "text"]
    )
    fig = px.scatter(
        viz_df,
        x=f"Attn Prob on Name",
        y=f"Dot w Name Embed",
        color="Name Type",
        hover_data=["text"],
        color_discrete_sequence=["rgb(114,255,100)", "rgb(201,165,247)"],
        title=f"How Strong {layer}.{head_idx} Writes in the Name Embed Direction Relative to Attn Prob",
    )

    if return_vals:
        return viz_df
    if return_fig:
        return fig
    else:
        fig.show()

In [ ]:
scatter_attention_and_contribution(model, (8, 9), ioi_dataset.toks, ioi_dataset.word_idx['end'], ioi_dataset.word_idx['IO'], ioi_dataset.word_idx['S2'], answer_residual_directions)

In [ ]:
top_k = 2
top_heads = torch.topk(-per_head_logit_diffs.flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, prompts[0], heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

In [ ]:
scatter_attention_and_contribution(model, (8, 9), prompts, io_positions, s_positions, answer_residual_directions)

## Activation Patching for Model Component Importance

### Attention Heads

In [ ]:
results = act_patch(
    model=model,
    orig_input=abc_dataset.toks,
    new_cache=clean_cache,
    patching_nodes=IterNode("z"), # iterating over all heads' output in all layers
    patching_metric=logit_diff_denoising_ioi,
    verbose=True,
)

In [ ]:
imshow_p(
    results['z'] * 100,
    title="Patching output of attention heads (corrupted -> clean)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=600,
    margin={"r": 100, "l": 100}
)

### Head Output by Component

In [ ]:
# iterating over all heads' output in all layers
results = act_patch(
    model=model,
    orig_input=abc_dataset.toks,
    new_cache=clean_cache,
    patching_nodes=IterNode(["z", "q", "k", "v", "pattern"]),
    patching_metric=logit_diff_denoising_ioi,
    verbose=True,
)

In [ ]:
assert results.keys() == {"z", "q", "k", "v", "pattern"}
#assert all([r.shape == (12, 12) for r in results.values()])

imshow_p(
    torch.stack(tuple(results.values())) * 100,
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Patching output of attention heads (corrupted -> clean)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=1500,
    margin={"r": 100, "l": 100}
)

### Residual Stream & Layer Outputs

In [ ]:
results = act_patch(
    model=model,
    orig_input=abc_dataset.toks,
    new_cache=clean_cache,
    patching_nodes=IterNode(["resid_pre", "attn_out", "mlp_out"], seq_pos="each"),
    patching_metric=logit_diff_denoising_ioi,
    verbose=True,
)

In [ ]:
assert results.keys() == {"resid_pre", "attn_out", "mlp_out"}
labels = [f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(ioi_dataset.toks[0]))]
imshow_p(
    torch.stack([r.T for r in results.values()]) * 100, # we transpose so layer is on the y-axis
    facet_col=0,
    facet_labels=["resid_pre", "attn_out", "mlp_out"],
    title="Patching at resid stream & layer outputs (corrupted -> clean)",
    labels={"x": "Sequence position", "y": "Layer", "color": "Logit diff variation"},
    x=labels,
    xaxis_tickangle=45,
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=1400,
    height=600,
    margin={"r": 100, "l": 100}
)

## Circuit Sketching

### First Level

#### Heads Influencing Logit Diff Directly

In [ ]:
model.cfg.use_attn_in = True

In [ ]:
path_patch_resid_post = path_patch(
    model,
    orig_input=ioi_dataset.toks,
    new_input=abc_dataset.toks,
    sender_nodes=IterNode('z'), # This means iterate over all heads in all layers
    receiver_nodes=Node('resid_post', 11), # This is resid_post at layer 11
    patching_metric=logit_diff_noising_ioi,
    verbose=True
)

In [ ]:
imshow_p(
    path_patch_resid_post['z'] * 100,
    title="Direct effect on logit diff (patch from head output -> final resid)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=600,
    margin={"r": 100, "l": 100}
)

In [ ]:
plot_attention_heads(-path_patch_resid_post['z'].cuda(), top_n=10, range_x=[0, 1.0])

In [ ]:
top_k = 1
DISPLAY_IDX = 0
top_heads = torch.topk(-path_patch_resid_post['z'].flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, model.to_string((ioi_dataset.toks[DISPLAY_IDX][:ioi_dataset.word_idx["end"][DISPLAY_IDX]+1])), heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

In [ ]:
#DE_NMH = [(8, 10), (8, 2)]
DE_S2I = [(8, 9)]
#DE_PUNC_CONJ = [(10, 1)]

In [ ]:
logit_diff_denoising_ioi_t = partial(logit_diff_denoising, dataset=ioi_dataset, return_tensor=True)
logit_diff_noising_ioi_t = partial(logit_diff_noising, dataset=ioi_dataset, return_tensor=True)

In [ ]:
ioi_metric = logit_diff_denoising_ioi
attn_head_pattern_all_pos_act_patch_results = patching.get_act_patch_attn_head_pattern_all_pos(model, abc_dataset.toks, ioi_cache, logit_diff_denoising_ioi_t)
# imshow(attn_head_pattern_all_pos_act_patch_results, 
#        yaxis="Layer", 
#        xaxis="Head", 
#        title="IOI Metric for 'attn_head_pattern' Activation Patching (All Pos)")

In [ ]:
attn_head_out_all_pos_act_patch_results = patching.get_act_patch_attn_head_out_all_pos(model, abc_dataset.toks, ioi_cache, logit_diff_denoising_ioi_t)
# imshow(attn_head_out_all_pos_act_patch_results, 
#        yaxis="Layer", 
#        xaxis="Head", 
#        title="IOI Metric for 'attn_head_out' Activation Patching (All Pos)")

In [ ]:
from utils.visualization import l_scatter
head_labels = [f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)]
l_scatter(
    x=utils.to_numpy(attn_head_pattern_all_pos_act_patch_results.flatten()), 
    y=utils.to_numpy(attn_head_out_all_pos_act_patch_results.flatten()), 
    hover_name = head_labels,
    xaxis="Attention Patch",
    yaxis="Output Patch",
    title="Scatter plot of output patching vs attention patching")

#### NMH Knockout

##### All Heads

In [ ]:
heads_to_ablate = DE_NMH

print(f"Heads to ablate: {heads_to_ablate}")
def ablate_top_head_hook(z: TT["batch", "pos", "head_index", "d_head"], hook, head_idx=0):
    z[:, :, head_idx, :] = 0
    return z
# Adds a hook into global model state
for layer, head in heads_to_ablate:
    ablate_head_hook = partial(ablate_top_head_hook, head_idx=head)
    model.blocks[layer].attn.hook_z.add_hook(ablate_head_hook)
# Runs the model, temporarily adds caching hooks and then removes *all* hooks after running, including the ablation hook.
ablated_logits, ablated_cache = model.run_with_cache(ioi_dataset.toks)
print(f"Original IOI Metric: {_logits_to_ave_logit_diff(clean_logits, ioi_dataset).item():.4f}")
print(f"Post ablation IOI Metric: {_logits_to_ave_logit_diff(ablated_logits, ioi_dataset).item()}")

In [ ]:
per_head_ablated_residual, labels = ablated_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_ablated_logit_diffs = residual_stack_to_logit_diff(per_head_ablated_residual, logit_diff_directions, prompts, ablated_cache)
per_head_ablated_logit_diffs = per_head_ablated_logit_diffs.reshape(model.cfg.n_layers, model.cfg.n_heads)
imshow_n(per_head_ablated_logit_diffs, labels={"x":"Head", "y":"Layer"}, zmin=-1.5, zmax=1.5, title="Post-Ablation Direct Logit Attribution of Heads")
l_scatter(y=per_head_logit_diffs.flatten(), x=per_head_ablated_logit_diffs.flatten(), hover_name=head_labels, range_x=(-3, 3), range_y=(-3, 3), xaxis="Ablated", yaxis="Original", title="Original vs Post-Ablation Direct Logit Attribution of Heads")

In [ ]:
exclusions = [(6, 6), (7, 9), (8, 9)] + [(9, 1), (9, 5)]
delta = per_head_ablated_logit_diffs - per_head_logit_diffs
for layer, head in exclusions:
    per_head_ablated_logit_diffs[layer, head] = 0

plot_attention_heads(
    per_head_ablated_logit_diffs/clean_logit_diff, 
    title="Logit Diff Contribution From Backup Heads", 
    top_n=15, 
    range_x=[0, 0.5]
)

##### Individual Heads

In [ ]:
# Get indices of all heads where the ablation had a positive effect
delta = per_head_ablated_logit_diffs - per_head_logit_diffs
backup_nmh_candidates = np.argwhere(delta.cpu().detach().numpy() > 0.05)
backup_nmh_candidates = [tuple(h) for h in backup_nmh_candidates]
backup_nmh_candidates = [h for h in backup_nmh_candidates if h not in exclusions]
print(f"Backup NMH Candidates: {backup_nmh_candidates}")
for l, h in backup_nmh_candidates:
    for layer, head in heads_to_ablate:
        ablate_head_hook = partial(ablate_top_head_hook, head_idx=head)
        model.blocks[layer].attn.hook_z.add_hook(ablate_head_hook)
    scatter_attention_and_contribution(model, (l, h), prompts, io_positions, s_positions, answer_residual_directions)

In [ ]:
attn_head_pattern_all_pos_act_patch_results['z'].shape

In [ ]:
top_k = 5
top_heads = torch.topk(-attn_head_pattern_all_pos_act_patch_results['z'].flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, prompts, heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

In [ ]:
# V-weighted version
plot_attention(
    model, 
    prompts[0],
    nmh_candidates,
    clean_cache,
    weighted=True)

### Contributors to DE Heads

#### Attention Out by Position

In [ ]:
results = path_patch(
    model,
    orig_input=ioi_dataset.toks,
    new_input=abc_dataset.toks,
    sender_nodes=IterNode(node_names=["attn_out"], seq_pos="each"),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in DE_S2I],
    patching_metric=logit_diff_noising_ioi,
    verbose=True,
)
results = einops.rearrange(results['attn_out'], "seq layer -> layer seq")

In [ ]:
imshow_n(
        results * 100,
        title=f"Direct effect on DE Heads' values",
        xaxis="Pos", 
        x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(ioi_dataset.toks[0]))],
        y=[f"Layer {layer}" for layer in range(model.cfg.n_layers)],
        width=1500,
        height=600,
    )

#####

#### Attention Out by Head

In [ ]:
results = path_patch(
    model,
    orig_input=ioi_dataset.toks,
    new_input=abc_dataset.toks,
    sender_nodes=IterNode("z"),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in DE_S2I],
    patching_metric=logit_diff_noising_ioi,
    verbose=True,
)

In [ ]:
imshow_p(
        results["z"][:10] * 100,
        title=f"Direct effect on NMH' queries",
        labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
        coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=700,
        margin={"r": 100, "l": 100}
    )

In [ ]:
plot_attention_heads(-results['z'].cuda(), top_n=10, range_x=[0, 1.0])

In [ ]:
top_k = 5
DISPLAY_IDX = 0
top_heads = torch.topk(-results['z'].flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, model.to_string((ioi_dataset.toks[DISPLAY_IDX][:ioi_dataset.word_idx["end"][DISPLAY_IDX]+1])), heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

### Second Level

#### Attention Pattern for Second-Level Heads

In [ ]:
second_level_positive_heads = [(6, 6), (7, 2), (7, 9)]
IE_S2I = second_level_positive_heads

tokens, attn, names = get_attn_head_patterns(model, model.to_string((ioi_dataset.toks[DISPLAY_IDX][:ioi_dataset.word_idx["end"][DISPLAY_IDX]+1])), second_level_positive_heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

#second_level_negative_heads = [(7, 8), (8, 10)]
#visualize_attention_patterns(torch.tensor([l*12+h for l, h in second_level_negative_heads]), title=f"Top Negative Second Level IOI Metric Heads")

In [ ]:
head_labels = [f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)]
l_scatter(
    x=utils.to_numpy(attn_head_v_all_pos_act_patch_results.flatten()), 
    y=utils.to_numpy(attn_head_out_all_pos_act_patch_results.flatten()), 
    xaxis="Value Patch",
    yaxis="Output Patch",
    #caxis="Layer",
    hover_name = head_labels,
    color=einops.repeat(np.arange(model.cfg.n_layers), "layer -> (layer head)", head=model.cfg.n_heads),
    range_x=(-1.5, 1.5),
    range_y=(-1.5, 1.5),
    title="Scatter plot of output patching vs value patching")

In [ ]:
s2i_candidates = [(6, 6), (7, 2), (7, 9), (8, 9)]
#s2i_candidates = [(8, 9)]

#### S2I Knockout

##### All Heads

In [ ]:
heads_to_ablate = s2i_candidates

print(f"Heads to ablate: {heads_to_ablate}")
def ablate_top_head_hook(z: TT["batch", "pos", "head_index", "d_head"], hook, head_idx=0):
    z[:, -1, head_idx, :] = 0
    return z
# Adds a hook into global model state
for layer, head in heads_to_ablate:
    ablate_head_hook = partial(ablate_top_head_hook, head_idx=head)
    model.blocks[layer].attn.hook_z.add_hook(ablate_head_hook)
# Runs the model, temporarily adds caching hooks and then removes *all* hooks after running, including the ablation hook.
ablated_logits, ablated_cache = model.run_with_cache(clean_tokens)
print(f"Original IOI Metric: {ioi_metric(clean_logits).item():.4f}")
print(f"Post ablation IOI Metric: {ioi_metric(ablated_logits).item()}")
#print(f"Direct Logit Attribution of top name mover head: {per_head_logit_diffs.flatten()[top_name_mover].item()}")
#print(f"Naive prediction of post ablation logit diff: {original_average_logit_diff - per_head_logit_diffs.flatten()[top_name_mover].item()}")

In [ ]:
per_head_ablated_residual, labels = ablated_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_ablated_logit_diffs = residual_stack_to_logit_diff(per_head_ablated_residual, ablated_cache)
per_head_ablated_logit_diffs = per_head_ablated_logit_diffs.reshape(model.cfg.n_layers, model.cfg.n_heads)
imshow(per_head_ablated_logit_diffs, labels={"x":"Head", "y":"Layer"})
l_scatter(y=per_head_logit_diffs.flatten(), x=per_head_ablated_logit_diffs.flatten(), hover_name=head_labels, range_x=(-3, 3), range_y=(-3, 3), xaxis="Ablated", yaxis="Original", title="Original vs Post-Ablation Direct Logit Attribution of Heads")

#### Path Patching for S2-Inhibition Candidates

In [ ]:
receiver_heads = second_level_positive_heads

results = path_patch(
    model,
    orig_input=ioi_dataset.toks,
    new_input=abc_dataset.toks,
    sender_nodes=IterNode("z"),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in s2i_candidates],
    patching_metric=logit_diff_noising_ioi,
    verbose=True,
)

In [ ]:
imshow_p(
        results["z"][:10] * 100,
        title=f"Direct effect on S2Is' values",
        labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
        coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=700,
        margin={"r": 100, "l": 100}
    )

In [ ]:
plot_attention_heads(-results['z'].cuda(), top_n=10, range_x=[0, 1.0])

### Third Level

#### Attention Patterns for Third-Level Heads

We have a mix of induction heads and duplicate token heads here, as well as two heads that focus on S2 at S2.

In [ ]:
third_level_positive_heads = [(4, 6), (4, 11)]
DISPLAY_IDX = 0
tokens, attn, names = get_attn_head_patterns(model, model.to_string((ioi_dataset.toks[DISPLAY_IDX][:ioi_dataset.word_idx["end"][DISPLAY_IDX]+1])), third_level_positive_heads)
#tokens, attn, names = get_attn_head_patterns(model, model.to_string((ioi_dataset.toks[DISPLAY_IDX][:ioi_dataset.word_idx["end"][DISPLAY_IDX]+1])), second_level_positive_heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

## Save the Circuit

In [ ]:
# define circuit
from collections import namedtuple
import pickle

CircuitComponent = namedtuple(
    "CircuitComponent", ["heads", "position", "receiver_type"]
)

circuit = {
    "name-movers": CircuitComponent(
        DE_NMH, -1, "hook_q"
    ),
    "s2-inhibition": CircuitComponent(s2i_candidates, ioi_dataset.word_idx["S2"], "hook_v"),
    # "duplicate-name": CircuitComponent([(7, 15), (9, 1)], 10, 'head_v'),
    # "induction": CircuitComponent([], 10, 'head_v')
}

# Specify a filename for saving the circuit dictionary
circuit_filename = 'results/circuits/pythia_160m_circuit.pkl'

# Save the circuit dictionary using pickle
with open(circuit_filename, 'wb') as f:
    pickle.dump(circuit, f)

In [ ]:

# save 410m circuit
circuit = {
    "name-movers": CircuitComponent(
        [(17, 10), (17, 6), (17, 11), (18, 0), (18, 8), (18, 13), (18, 14)],
        -1,
        "hook_q",
    ),
    "s2-inhibition": CircuitComponent(
        [(11, 4), (13, 1), (13, 5), (16, 0)], 10, "hook_v"
    ),
    # "duplicate-name": CircuitComponent([], 10, 'head_v'),
    # "induction": CircuitComponent([], 10, 'head_v')
}

In [ ]:
# save 1.4b circuit
circuit = {
    "name-movers": CircuitComponent(
        [(12, 15), (13, 1), (13, 6), (15, 15), (16, 13), (17, 7)], -1, "hook_q"
    ),
    "s2-inhibition": CircuitComponent([(10, 7)], 10, "hook_v"),
    # "duplicate-name": CircuitComponent([(7, 15), (9, 1)], 10, 'head_v'),
    # "induction": CircuitComponent([], 10, 'head_v')
}

In [ ]:
# save 2.8b circuit
circuit = {
    "name-movers": CircuitComponent(
        [(12, 15), (13, 1), (13, 6), (15, 15), (16, 13), (17, 7)], -1, "hook_q"
    ),
    "s2-inhibition": CircuitComponent([(10, 7)], 10, "hook_v"),
    # "duplicate-name": CircuitComponent([(7, 15), (9, 1)], 10, 'head_v'),
    # "induction": CircuitComponent([], 10, 'head_v')
}

In [ ]:
circuit_file = "pythia_1-4b_circuit.pkl"
circuit_root = "results/circuits/"
with open(circuit_root + circuit_file, 'rb') as f:
    circuit = pickle.load(f)

In [ ]:
res = torch.load(f"results/pythia-1.4b-no-dropout/value_perf.pt")

In [ ]:
res[18, 12]

In [ ]:
model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-1.4b",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
)
model.set_use_hook_mlp_in(True)

In [ ]:
model.cfg